In [38]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
      --------------------------------------- 0.2/12.9 MB 4.6 MB/s eta 0:00:03
     --- ------------------------------------ 1.0/12.9 MB 11.1 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.9 MB 20.2 MB/s eta 0:00:01
     --------- ------------------------------ 3.2/12.9 MB 22.7 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.9 MB 28.9 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.9 MB 30.5 MB/s eta 0:00:01
     ------------------------------ --------- 9.9/12.9 MB 31.6 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.9 MB 36.4 MB/s eta 0:00:01
     ----------------------------------- --- 11.6/12.9 MB 32.7 MB/s eta 0:00:01
     --------------------------------------  12.9/12.9 MB 32.8 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 27.3 MB/s eta 0:00:00
✔ Download and installation successful
You can no


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
import PyPDF2
import re
import es_core_news_sm

In [40]:
# Definimos la lista de stopwords en español
stopwords_es = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 
    'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 
    'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 
    'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 
    'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 
    'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 
    'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 
    'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 
    'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 
    'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 
    'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 
    'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'es'
]

In [44]:
# Carga del modelo de nlp
nlp = es_core_news_sm.load()  

In [27]:
def procesar_texto(text: str) -> list[str]:
    """
    Función que a partir de un texto ingresado por argumento, se lo normaliza, elimina stopwords y lematiza. 
    La salida será una lista con sus palabras claves.
    """

    # Eliminación de mayúsculas
    text = text.lower()
    
    # Eliminación puntuación 
    text = re.sub(r'[^\w\s]', '', text)
    
    # Lematización del texto     
    #nlp = es_core_news_sm.load()    
    doc = nlp(text)
    lemmas = [tok.lemma_.lower() for tok in doc]
    
    # Filtramos las palabras para eliminar las stopwords
    palabras_filtradas = [palabra for palabra in lemmas if palabra not in stopwords_es]
    
    return palabras_filtradas

In [43]:
"""
Celda donde se carga el pdf Lunfardo y se extrae su contenido, siendo la
palabra del lunfardo la clave del diccionario y su definición el valor de la misma.
""" 

# Abre el archivo en modo binario de lectura ('rb')
with open('Lunfardo.pdf', 'rb') as archivo:
    # Crea un objeto PdfFileReader
    lector = PyPDF2.PdfReader(archivo)

# Inicializa una cadena vacía para almacenar el texto
    texto = ''

    # Itera sobre todas las páginas del PDF
    for i in range(1,61):
        # Obtiene la página
        pagina = lector.pages[i]

        # Extrae el texto de la página y lo añade a la cadena de texto
        texto += pagina.extract_text ()

# Diccionario para almacenar las palabras y definiciones
dict_lunfardo = {}

regex = r"(\w+(?:\(se\))?)\s*:\s*(.*?)(?=\n\w+|$)"
resultado = re.findall(regex, texto, re.DOTALL)
dict_lunfardo = {clave: valor.strip() for clave, valor in resultado}

# Limpio y lematizo las definiciones del diccionario
for lunfardo in dict_lunfardo: 
    dict_lunfardo[lunfardo] = procesar_texto(dict_lunfardo[lunfardo])
    try:
        dict_lunfardo[lunfardo].remove(' ')
    except: 
        continue


print(dict_lunfardo)

{'Abanico': ['agente', 'policial', 'delator', 'soplón', 'puerto'], 'Abarajar': ['parar', 'cuchillo', 'golpe', 'adversario', 'agarrar', 'asir'], 'Abatatar(se)': ['turbar él', 'perder', 'serenidad', 'libre', 'uso', 'facultad'], 'Abisagrar(se)': ['casar él'], 'Abocar(se)': ['lanzar él', 'precipitadamente'], 'Abombar': ['aturdir', 'turbar', 'corromper él', 'carne', 'corromper él', 'agua'], 'Abotonar': ['cerrar'], 'Abrir(se)': ['apartar él', 'desviar él', 'desistir', 'separar él', 'empresa'], 'Abrochar': ['cerrar'], 'Acabar': ['eyacular'], 'Academia': ['salón', 'atendido', 'camarera', 'beber', 'bailar'], 'Acamalar': ['ajobar', 'cargar', 'alguno', 'cosa', 'agarrar', ' ', 'asir', 'tomar', 'ahorra', 'r'], 'Aceitar': ['sobornar'], 'Aceite': ['expresión', 'tomar él', 'aceite', 'retirar él', 'huir'], 'Aceitosa': ['cabeza'], 'Acomodar': ['aplicar', 'dar', 'propinar', 'proporcionar', 'situación'], 'Acoyarar(se)': ['unir él', 'alguien', 'matrimonio', 'concubinato'], 'Achacado': ['achacoso', 'padecer

In [60]:
def buscador(dict_lunfardo : dict, text: str) -> None:
    """ 
    Funcíon que muestra en pantalla los tres lunfardos que más se asemejan a la definición ingresada por el usuario.
    """
    # Lematizo la definición ingresada
    texto_lemma = procesar_texto(text)

    # Creo un nuevo diccionario, donde la clave será el lunfardo, y su valor el porcentaje de semejanza con la definición.
    porcentajes = {}

    # Itero sobre el diccionario de lunfardos, buscando el que más se adecue a la definición.
    for lunfardo in dict_lunfardo:
        cont_semejanza = 0        
        for palabra in texto_lemma: 
            if palabra in dict_lunfardo[lunfardo]:
                cont_semejanza += 1 
        
        porcentajes[lunfardo] = cont_semejanza
  
    # Ordeno de forma descendiente el diccionario
    porcentajes_ordenado = dict(sorted(porcentajes.items(), key=lambda item: item[1], reverse=True))

    # Muestro en pantalla los 3 lunfardos con mayor semejanza
    tres_primeros = dict(list(porcentajes_ordenado.items())[:3])

    print(tres_primeros)

In [62]:
# Codigo cliente
buscador(dict_lunfardo, 'Persona que es tonta, con pocas capacidades mentales')
dict_lunfardo

{'Bachicha': 2, 'Ñoqui': 2, 'Oligo': 2}


{'Abanico': ['agente', 'policial', 'delator', 'soplón', 'puerto'],
 'Abarajar': ['parar', 'cuchillo', 'golpe', 'adversario', 'agarrar', 'asir'],
 'Abatatar(se)': ['turbar él',
  'perder',
  'serenidad',
  'libre',
  'uso',
  'facultad'],
 'Abisagrar(se)': ['casar él'],
 'Abocar(se)': ['lanzar él', 'precipitadamente'],
 'Abombar': ['aturdir',
  'turbar',
  'corromper él',
  'carne',
  'corromper él',
  'agua'],
 'Abotonar': ['cerrar'],
 'Abrir(se)': ['apartar él',
  'desviar él',
  'desistir',
  'separar él',
  'empresa'],
 'Abrochar': ['cerrar'],
 'Acabar': ['eyacular'],
 'Academia': ['salón', 'atendido', 'camarera', 'beber', 'bailar'],
 'Acamalar': ['ajobar',
  'cargar',
  'alguno',
  'cosa',
  'agarrar',
  ' ',
  'asir',
  'tomar',
  'ahorra',
  'r'],
 'Aceitar': ['sobornar'],
 'Aceite': ['expresión', 'tomar él', 'aceite', 'retirar él', 'huir'],
 'Aceitosa': ['cabeza'],
 'Acomodar': ['aplicar', 'dar', 'propinar', 'proporcionar', 'situación'],
 'Acoyarar(se)': ['unir él', 'alguien', '